In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 13 #Referente aos 14 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] 
data = data.drop('Acre - Produção de Cimento (t)', axis=1)
data = data.fillna(-1)
data

,Ano,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Consumo de Cimento (t)
0,2008,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.691749,0.773119,0.647836,0.657765,...,2.868108,11.760983,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,119.145000
1,2009,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.695139,0.777398,0.653540,0.658635,...,2.305885,9.472585,27.667957,984.971935,1299.760655,25.376339,21.219990,79.673793,1.436413,121.146000
2,2010,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.698459,0.781539,0.659233,0.659404,...,1.958246,9.332485,30.587558,985.037587,1305.345781,25.288855,17.738219,70.049719,1.246421,174.705000
3,2011,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.701724,0.785565,0.664930,0.660077,...,2.079088,10.985689,34.156125,985.282207,1317.710216,25.242893,16.188303,66.211701,1.234468,178.322000
4,2012,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.704955,0.789498,0.670645,0.660660,...,2.183282,8.159014,31.678793,985.498854,1297.603435,25.454002,17.919229,70.364429,1.472205,173.494000
5,2012,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.709974,0.795847,0.678860,0.661919,...,2.183282,8.159014,31.678793,985.498854,1297.603435,25.454002,17.919229,70.364429,1.472205,173.494000
6,2013,11.122668,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.710721,0.796251,0.681349,0.661246,...,2.403445,7.913561,25.635353,985.492097,1244.679422,26.085119,20.000566,73.871198,1.665816,193.672000
7,2014,11.666939,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.711115,0.796188,0.683294,0.660516,...,2.748397,10.350442,23.987188,985.694959,1230.934473,26.457083,21.165984,76.068571,1.660711,171.709286
8,2015,12.139670,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.720255,0.808026,0.697137,0.663153,...,2.501529,12.454398,28.288853,986.021421,1278.252041,26.043363,21.489384,78.704494,1.603219,151.486629
9,2016,12.458072,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.720460,0.807937,0.698785,0.662167,...,2.073464,13.111908,31.329110,985.952489,1333.799567,25.936106,21.376188,78.087996,1.551043,134.223086


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.514659,7.755114e+06,422290.202070,9.424555,6.724366e+06,0.691749,0.773119,0.647836,0.657765,5.483335e+08,...,-7723.574973,2.868108,11.760983,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,8.542386,7.805166e+06,424173.603186,9.409042,6.722105e+06,0.695139,0.777398,0.653540,0.658635,5.302764e+08,...,-8961.781709,2.305885,9.472585,27.667957,984.971935,1299.760655,25.376339,21.219990,79.673793,1.436413
2,8.514295,7.991788e+06,416829.675918,9.394384,6.762279e+06,0.698459,0.781539,0.659233,0.659404,5.046425e+08,...,-9895.682385,1.958246,9.332485,30.587558,985.037587,1305.345781,25.288855,17.738219,70.049719,1.246421
3,8.500790,8.164220e+06,410997.875281,9.386906,6.795421e+06,0.701724,0.785565,0.664930,0.660077,4.854622e+08,...,-9989.170714,2.079088,10.985689,34.156125,985.282207,1317.710216,25.242893,16.188303,66.211701,1.234468
4,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.704955,0.789498,0.670645,0.660660,4.867661e+08,...,-9429.326328,2.183282,8.159014,31.678793,985.498854,1297.603435,25.454002,17.919229,70.364429,1.472205
5,10.334544,8.325627e+06,407346.950074,9.383420,6.827686e+06,0.709974,0.795847,0.678860,0.661919,4.867661e+08,...,-9429.326328,2.183282,8.159014,31.678793,985.498854,1297.603435,25.454002,17.919229,70.364429,1.472205
6,11.122668,8.462722e+06,403688.428174,9.371494,6.849508e+06,0.710721,0.796251,0.681349,0.661246,5.225848e+08,...,-7157.441850,2.403445,7.913561,25.635353,985.492097,1244.679422,26.085119,20.000566,73.871198,1.665816
7,11.666939,8.560896e+06,400704.121188,9.343267,6.852462e+06,0.711115,0.796188,0.683294,0.660516,5.676413e+08,...,-3453.461327,2.748397,10.350442,23.987188,985.694959,1230.934473,26.457083,21.165984,76.068571,1.660711
8,12.139670,8.597116e+06,391942.316578,9.284504,6.825760e+06,0.720255,0.808026,0.697137,0.663153,5.179392e+08,...,-1442.919387,2.501529,12.454398,28.288853,986.021421,1278.252041,26.043363,21.489384,78.704494,1.603219
9,12.458072,8.620855e+06,389049.090551,9.247193,6.809907e+06,0.720460,0.807937,0.698785,0.662167,4.524144e+08,...,-3128.377228,2.073464,13.111908,31.329110,985.952489,1333.799567,25.936106,21.376188,78.087996,1.551043


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     121.146000
1     174.705000
2     178.322000
3     173.494000
4     173.494000
5     193.672000
6     171.709286
7     151.486629
8     134.223086
9     150.491000
10    148.696000
11    153.806000
12    178.033000
13    189.357000
14    202.321000
15           NaN
Name: Acre - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)  
train_input

,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.369699,-1.923882,1.556182,1.271411,-1.892655,-1.778753,-1.826148,-1.743517,-2.007377,1.261808,...,-0.500917,2.126822,1.059506,-3.210438,-3.464000,-3.441868,-3.444828,-3.285261,-3.405684,-3.312266
1,-1.351990,-1.759324,1.713749,1.057077,-1.949089,-1.442141,-1.461631,-1.430606,-1.464907,0.862198,...,-0.772083,0.115581,-0.002029,-0.062880,0.282570,0.269902,0.190200,0.474004,0.573573,0.315895
2,-1.369932,-1.145760,1.099350,0.854550,-0.946590,-1.112597,-1.108885,-1.118275,-0.985347,0.294908,...,-0.976607,-1.128027,-0.067018,0.257673,0.282820,0.285839,0.178144,-0.115056,0.098862,0.032970
3,-1.378557,-0.578848,0.611457,0.751227,-0.119556,-0.788349,-0.765962,-0.805744,-0.565546,-0.129560,...,-0.997081,-0.695740,0.699865,0.649479,0.283749,0.321121,0.171809,-0.377277,-0.090449,0.015169
4,-0.207329,-0.048182,0.306017,0.703072,0.685576,-0.467626,-0.430910,-0.492233,-0.202352,-0.100704,...,-0.874475,-0.323007,-0.611365,0.377484,0.284573,0.263746,0.200903,-0.084432,0.114386,0.369193
5,-0.207329,-0.048182,0.306017,0.703072,0.685576,0.030642,0.109939,-0.041571,0.583129,-0.100704,...,-0.874475,-0.323007,-0.611365,0.377484,0.284573,0.263746,0.200903,-0.084432,0.114386,0.369193
6,0.296050,0.402550,-0.000057,0.538295,1.230104,0.104885,0.144345,0.094934,0.163170,0.691978,...,-0.376934,0.464584,-0.725225,-0.286047,0.284547,0.112725,0.287880,0.267697,0.287358,0.657507
7,0.643679,0.725323,-0.249727,0.148305,1.303822,0.143948,0.139025,0.201672,-0.291786,1.689098,...,0.434236,1.698578,0.405188,-0.467005,0.285318,0.073504,0.339142,0.464867,0.395744,0.649906
8,0.945615,0.844403,-0.982746,-0.663597,0.637512,1.051456,1.147464,0.961023,1.352513,0.589170,...,0.874543,0.815461,1.381165,0.005290,0.286558,0.208526,0.282125,0.519581,0.525762,0.564292
9,1.148979,0.922452,-1.224796,-1.179102,0.241919,1.071813,1.139921,1.051471,0.737544,-0.860922,...,0.505429,-0.715858,1.686169,0.339091,0.286296,0.367033,0.267344,0.500430,0.495353,0.486594


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
# validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
# validation_input = dfToInputRNN(validation_input)
# validation_input

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     121.146000
1     174.705000
2     178.322000
3     173.494000
4     173.494000
5     193.672000
6     171.709286
7     151.486629
8     134.223086
9     150.491000
10    148.696000
11    153.806000
12    178.033000
Name: Acre - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
# validation_target = target_data.iloc[split_index - 1:-3]
# validation_target

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
13,0.75019,0.422372,-0.614309,-1.173545,-0.477662,0.791694,0.610405,0.928926,0.186972,1.163167,...,1.038882,-1.522321,-2.313421,0.680972,0.293183,0.339744,0.682904,0.673701,0.048441,-0.613893


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.75019046,  0.42237244, -0.61430895, -1.17354496,
         -0.47766234,  0.79169443,  0.61040493,  0.92892634,
          0.18697239,  1.16316662,  2.6648078 , -0.28415403,
          0.78842482,  2.70456197, -0.13731248,  1.0620003 ,
          0.14874796, -0.17746697,  1.94394596,  2.11975022,
         -2.30353113,  1.50318835,  0.92591867,  0.21976702,
          0.38034066,  1.07012252,  2.38027465,  1.18721136,
          0.68454647,  0.96357222,  1.49963583,  1.10064062,
          1.54938258, -1.601903  ,  1.82071202,  1.75021424,
          1.09338904,  0.63049382, -1.89207081, -2.1907463 ,
         -2.64430562,  0.34649211, -1.64566109,  0.29105292,
         -0.8938401 ,  1.60067003,  1.08600407,  1.57190372,
          1.48403903,  1.96565138, -2.59491426,  2.94159734,
          1.66925951,  0.0371629 ,  1.62436238,  1.59912894,
          1.82219669,  1.96888178,  1.62404736,  1.59415473,
          1.92575804,  2.73371511,  1.06901998,  1.0388819 ,
         -1.5223207 , -2

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

13    189.357
Name: Acre - Consumo de Cimento (t), dtype: float64

In [29]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=1000, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(592, activation='tanh', return_sequences=True, input_shape=(train_input.shape[1], 
                                                                                         train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=5000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [30]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, 0, seed)

        current_loss = min(history.history['loss'])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [31]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[797950828, 1546085163, 1943605974, 1768281739, 2728393309, 2172910185, 1942859836, 1592464629, 1351095117, 939494481, 3506784771, 888451179, 3685148622, 1625575914, 2178473043, 3682420205, 2078668894, 749097323, 1646322377, 1918659770, 787816461, 3056901762, 4164134211, 3526093835, 2736202719, 3738680515, 529751079, 2145533329, 3414127755, 1725373489, 1324388357, 1691334399, 4047654372, 1728168801, 2758028599, 582041760, 2565119530, 1080466134, 3903619677, 2765029918, 3719231594, 3951320588, 4110833022, 3116264350, 2108107742, 2083336171, 466567181, 3351559270, 2067384097, 3660198050, 1726968277, 1168172214, 2567168495, 3139582485, 3036548433, 3820685314, 846122622, 1554368744, 415307735, 3964256952, 2990844777, 4081093899, 3703764593, 2017784329, 3616623118, 207798711, 2108824843, 783173365, 902903132, 3810428714, 775590768, 2316594787, 449882948, 2610464160, 2962070065, 4039673749, 3510954195, 432729304, 269230661, 926917298, 3355634482, 403791394, 1762667832, 2853612215, 2958326148

2023-09-02 11:22:16.772730: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 11:22:16.776357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 11:22:16.777660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 0.07423786073923111
winner_seed: 797950828


Step: 1 ___________________________________________


2023-09-02 11:23:16.977890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 11:23:16.979561: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 11:23:16.980700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 0.08077442646026611


Step: 2 ___________________________________________


...

loss: 0.11130332201719284


Step: 98 ___________________________________________


2023-09-02 13:07:35.344549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:07:35.345694: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:07:35.346840: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 0.07106198370456696


Step: 99 ___________________________________________


2023-09-02 13:08:24.496170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:08:24.496970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:08:24.497545: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

loss: 0.10236186534166336


final_seed: 3685148622


In [32]:
trained_model, history = lstm_model(reshaped_train_input, 
                                    train_target, want_verbose=1, seed=winner_seed)

2023-09-02 13:09:12.148160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:09:12.149214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:09:12.150194: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/5000


2023-09-02 13:09:12.432379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:09:12.433915: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:09:12.435282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 4s 4s/step - loss: 26554.6152
Epoch 2/5000
1/1 [==============================] - 0s 13ms/step - loss: 25045.0645
Epoch 3/5000
1/1 [==============================] - 0s 12ms/step - loss: 19650.8242
Epoch 4/5000
1/1 [==============================] - 0s 11ms/step - loss: 8145.4253
Epoch 5/5000
1/1 [==============================] - 0s 11ms/step - loss: 1472.7124
Epoch 6/5000
1/1 [==============================] - 0s 10ms/step - loss: 444.4651
Epoch 7/5000
1/1 [==============================] - 0s 10ms/step - loss: 356.4379
Epoch 8/5000
1/1 [==============================] - 0s 10ms/step - loss: 348.4395
Epoch 9/5000
1/1 [==============================] - 0s 11ms/step - loss: 319.2108
Epoch 10/5000
1/1 [==============================] - 0s 10ms/step - loss: 301.1998
Epoch 11/5000
1/1 [==============================] - 0s 10ms/step - loss: 252.5208
Epoch 12/5000
1/1 [==============================] - 0s 10ms/step - loss: 205.9612
Epoch 13/5000
1/1 [=

1/1 [==============================] - 0s 10ms/step - loss: 30.8211
Epoch 102/5000
1/1 [==============================] - 0s 11ms/step - loss: 27.3370
Epoch 103/5000
1/1 [==============================] - 0s 9ms/step - loss: 27.7880
Epoch 104/5000
1/1 [==============================] - 0s 10ms/step - loss: 43.0431
Epoch 105/5000
1/1 [==============================] - 0s 11ms/step - loss: 28.0554
Epoch 106/5000
1/1 [==============================] - 0s 10ms/step - loss: 24.3380
Epoch 107/5000
1/1 [==============================] - 0s 9ms/step - loss: 31.9769
Epoch 108/5000
1/1 [==============================] - 0s 10ms/step - loss: 30.4360
Epoch 109/5000
1/1 [==============================] - 0s 11ms/step - loss: 40.3346
Epoch 110/5000
1/1 [==============================] - 0s 11ms/step - loss: 27.3940
Epoch 111/5000
1/1 [==============================] - 0s 10ms/step - loss: 23.2971
Epoch 112/5000
1/1 [==============================] - 0s 10ms/step - loss: 28.3922
Epoch 113/5000
1/1 [=

...

1/1 [==============================] - 0s 8ms/step - loss: 0.1236
Epoch 4571/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.3257
Epoch 4572/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.3747
Epoch 4573/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.2510
Epoch 4574/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5189
Epoch 4575/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.3721
Epoch 4576/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5817
Epoch 4577/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5243
Epoch 4578/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.3305
Epoch 4579/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.7431
Epoch 4580/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5724
Epoch 4581/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.1799
Epoch 4582/5000
1/1 [===========

1/1 [==============================] - 0s 8ms/step - loss: 0.3276
Epoch 4670/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.6236
Epoch 4671/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5563
Epoch 4672/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.2914
Epoch 4673/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.3819
Epoch 4674/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.4846
Epoch 4675/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.4524
Epoch 4676/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.3310
Epoch 4677/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.4744
Epoch 4678/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5360
Epoch 4679/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.4823
Epoch 4680/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.2534
Epoch 4681/5000
1/1 [=======

1/1 [==============================] - 0s 9ms/step - loss: 1.1439
Epoch 4769/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.9705
Epoch 4770/5000
1/1 [==============================] - 0s 9ms/step - loss: 1.7136
Epoch 4771/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.7681
Epoch 4772/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.7409
Epoch 4773/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5680
Epoch 4774/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.2297
Epoch 4775/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5404
Epoch 4776/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.7523
Epoch 4777/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.4629
Epoch 4778/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.2864
Epoch 4779/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.3530
Epoch 4780/5000
1/1 [======

1/1 [==============================] - 0s 8ms/step - loss: 0.4470
Epoch 4868/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.3289
Epoch 4869/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.4596
Epoch 4870/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.4993
Epoch 4871/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.1245
Epoch 4872/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.2402
Epoch 4873/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.4189
Epoch 4874/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.9622
Epoch 4875/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.4807
Epoch 4876/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.3397
Epoch 4877/5000
1/1 [==============================] - 0s 10ms/step - loss: 1.2179
Epoch 4878/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.9381
Epoch 4879/5000
1/1 [========

1/1 [==============================] - 0s 12ms/step - loss: 0.5154
Epoch 4967/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.7616
Epoch 4968/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.6477
Epoch 4969/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.7148
Epoch 4970/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.6926
Epoch 4971/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.6346
Epoch 4972/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.6052
Epoch 4973/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.7842
Epoch 4974/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.3359
Epoch 4975/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.6471
Epoch 4976/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5021
Epoch 4977/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.6102
Epoch 4978/5000
1/1 [=====

In [33]:
prediction = trained_model.predict(reshaped_test_input)[0][0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

2023-09-02 13:10:07.361777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:10:07.362619: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:10:07.363265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 328ms/step
[189.357](test_target) - [178.69551](prediction) = 10.661489135742187


In [34]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.05630364409946391

In [35]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = lstm_model(dfToInputRNN(train_input), 
                                       dfToInputRNN(train_target), want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0][0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [36]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Acre - Desemprego,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH,Acre - IDH Longevidade,Acre - IDH Educacao,Acre - IDH Renda,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.693210,-0.938459,0.382853,1.236143,-0.644859,-1.229053,-1.23139,-1.225123,-1.249086,1.148205,...,1.277766,1.308949,1.412346,-1.409259,-1.414214,-1.414204,-1.414208,-1.399129,-1.405755,-1.410735
1,1.414123,-0.446997,0.987585,-0.023123,-0.767579,0.008662,0.01340,0.000757,0.050227,0.140893,...,-0.114016,-0.190804,-0.643250,0.602204,0.707036,0.702558,0.710628,0.877970,0.836631,0.791222
2,-0.720913,1.385456,-1.370438,-1.213019,1.412438,1.220391,1.21799,1.224366,1.198859,-1.289098,...,-1.163750,-1.118144,-0.769096,0.807055,0.707177,0.711646,0.703580,0.521159,0.569123,0.619513


train_target:


,Acre - Consumo de Cimento (t)
0,121.146
1,174.705
2,178.322


test_input:


...

test_target:


,Acre - Consumo de Cimento (t)
13,189.357


2023-09-02 13:15:30.188570: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-02 13:15:30.189363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-02 13:15:30.189951: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 320ms/step
Error: 10.661489135742187




[178.3372039794922,
 173.410888671875,
 173.5752410888672,
 192.60621643066406,
 169.4825439453125,
 150.0540313720703,
 133.6951141357422,
 153.73956298828125,
 148.5023651123047,
 161.73130798339844,
 178.6955108642578]

In [37]:
display(targets)
display(predictions)

[173.494,
 173.494,
 193.672,
 171.7092857142857,
 151.48662857142855,
 134.2230857142857,
 150.491,
 148.696,
 153.80599999999998,
 178.033,
 189.357]

[178.3372039794922,
 173.410888671875,
 173.5752410888672,
 192.60621643066406,
 169.4825439453125,
 150.0540313720703,
 133.6951141357422,
 153.73956298828125,
 148.5023651123047,
 161.73130798339844,
 178.6955108642578]

In [38]:
mae = mean_absolute_error(predictions, targets)
mae

12.168466441761366

In [39]:
porcentage = mae/np.mean(targets)
porcentage

0.07360787899850259